In [16]:
import pandas as pd

df = pd.read_csv('coco/category_counts.csv')
coco_classes = dict(zip(df['category'].str.lower(), df['count']))

df = pd.read_csv('lvis/category_counts.csv')
lvis_classes = dict(zip(df['category'].str.lower(), df['count']))

df = pd.read_csv('objects365/category_counts.csv')
obj365_classes = dict(zip(df['category'].str.lower(), df['count']))

df = pd.read_csv('imagenet_boxes/category_counts.csv')
imagenet_boxes_classes = dict(zip(df['category'].str.lower(), df['count']))

df = pd.read_csv('imagenet_21k/category_counts.csv')
imagenet_21k_classes = dict(zip(df['category'].str.lower(), df['count']))

df = pd.read_csv('open_images/box_annotations_class_counts.csv')
open_images_boxes_classes = dict(zip(df['class'].str.lower(), df['count']))

df = pd.read_csv('open_images/point_annotations_class_counts.csv')
open_images_point_classes = dict(zip(df['class'].str.lower(), df['count']))

df = pd.read_csv('open_images/human_verified_image_labels_class_counts.csv')
open_images_clf_human_verified_classes = dict(zip(df['class'].str.lower(), df['count']))

df = pd.read_csv('open_images/machine_generated_image_labels_class_counts.csv')
open_images_clf_machine_generated_classes = dict(zip(df['class'].str.lower(), df['count']))

In [17]:
import requests

def request_solr(core, field, query):
    url = f'http://localhost:8983/solr/{core}/select' \
    f'?defType=edismax&df={field}&q={query}&start=0&rows=50'
    result = requests.get(url).json()
    num_found = result['response']['numFound']
    first_found = [x[field] for x in result['response']['docs']]
    return num_found, first_found

def query_refcoco_descriptions(title):
    num_found, first_found = request_solr('refcoco', 'Text', title)
    return num_found, f'{num_found/142210*100:g}%', first_found

def query_open_images_descriptions(title):
    num_found, first_found = request_solr('open_images_narratives', 'Text', title)
    return num_found, f'{num_found/507444*100:g}%', first_found

def query_visual_genome_descriptions(title):
    num_found, first_found = request_solr('visual_genome', 'Text', title)
    return num_found, f'{num_found/9211063*100:g}%', first_found

def query_laion400m_subset_descriptions(title):
    num_found, first_found = request_solr('laion400m_subset', 'Text', title)
    return num_found, f'{num_found/12933524*100:g}%', first_found

def query_laion_coco_subset_descriptions(title):
    num_found, first_found = request_solr('laion_coco', 'Text', title)
    return num_found, f'{num_found/5177571*100:g}%', first_found

In [18]:
query_laion_coco_subset_descriptions('panda -logo -icon -poster')

(1726,
 '0.0333361%',
 [["an image of a panda with the words'angry panda '"],
  ['A panda bear with the words, designer panda bear.'],
  ['Panda Run - Screenshote game.'],
  ['Panda Dom Advanced Software'],
  ['Panda Antivirus Pro 2012'],
  ['Panda Global Protection 2013.'],
  ['Panda Internet Security 2014'],
  ['Panda Slot Casino, Vegas.'],
  ['Panda tips and tricks.'],
  ['A panda love postcard.'],
  ['Panda Antivirus Pro 2014'],
  ['The Panda business model.'],
  ['A panda bear with the words keep calm and love panda.'],
  ['Three posters with a panda bear and a panda bear saying.'],
  ['a panda bear with the words destroy racism be like a panda'],
  ['a panda express chinese kitchen'],
  ['A Panda Press video guide.'],
  ['A KUNG PANDA 4 trailer.'],
  ['The China Panda silver coin.'],
  ['The china gold panda coin'],
  ['A panda bear coloring pages.'],
  ['Save the Panda phone case.'],
  ['The Royal Panda Casino website.'],
  ['Kung Panda the Legend Warriors.'],
  ['Penguin and Pa

In [10]:
def summary_report(synset):
    for dataset_name, instances_dict in {
        'COCO': coco_classes,
        'LVIS': lvis_classes,
        'Objects365': obj365_classes,
        'ImageNet boxes': imagenet_boxes_classes,
        'OpenImages boxes': open_images_boxes_classes,
        'OpenImages points': open_images_point_classes,
        'OpenImages clf (human verified)': open_images_clf_human_verified_classes,
        'OpenImages clf (machine generated)': open_images_clf_machine_generated_classes,
    }.items():
        print(dataset_name)
        for cls_name, count in instances_dict.items():
            for syn in synset:
                if (' ' + syn + ' ') in (' ' + cls_name + ' '):
                    percent = 100*count/sum(instances_dict.values())
                    print(f'\t({count}, {percent:.5f}%) {cls_name}')

    for query_fn, descr, show_abs_count in [
        (query_refcoco_descriptions, 'RefCOCO descriptions', True),
        (query_open_images_descriptions, 'OpenImages descriptions', True),
        (query_visual_genome_descriptions, 'Visual Genome descriptions', True),
        (query_laion400m_subset_descriptions, 'LAION 400M descriptions', False),
        (query_laion_coco_subset_descriptions, 'LAION COCO descriptions', False),
    ]:
        num_found, percent_found, found = query_fn(
            ' '.join(['"' + x + '"' for x in synset])
        )
        if show_abs_count:
            print(f'{descr}: {num_found} ({percent_found})')
        else:
            print(f'{descr}: {percent_found})')
        for i, f in enumerate(found[:5]):
            print(f'\tExample {i+1}: {f[0][:90]}')

summary_report(['badger', 'meles meles'])

COCO
LVIS
Objects365
ImageNet boxes
	(584, 0.03940%) badger
OpenImages boxes
OpenImages points
	(50, 0.00009%) badger
	(3, 0.00001%) european badger
OpenImages clf (human verified)
	(1191, 0.00203%) badger
	(971, 0.00165%) honey badger
	(3, 0.00001%) european badger
OpenImages clf (machine generated)
	(78, 0.00005%) badger
	(1, 0.00000%) honey badger
RefCOCO descriptions: 1 (0.000703185%)
	Example 1: 6 badger market
OpenImages descriptions: 3 (0.000591198%)
	Example 1: In this picture we can see badger on the grass.
	Example 2: In this image we can see a badger on the land. On the backside we can see a wall.
	Example 3: In this image I can see the toys which are in black, red, blue and yellow color. These are
Visual Genome descriptions: 20 (0.00021713%)
	Example 1: badger murphy
	Example 2: badger; wisconsinite.n.01
	Example 3: badger; wisconsinite.n.01
	Example 4: badger; wisconsinite.n.01
	Example 5: badger; wisconsinite.n.01
LAION 400M descriptions: 0.0109715%)
	Example 1: Badger Cu

In [11]:
summary_report(['panda', 'ailuropoda melanoleuca'])

COCO
LVIS
Objects365
ImageNet boxes
OpenImages boxes
	(882, 0.00604%) panda
	(423, 0.00290%) red panda
OpenImages points
	(6929, 0.01254%) panda
	(4727, 0.00855%) red panda
OpenImages clf (human verified)
	(1877, 0.00319%) panda
	(1241, 0.00211%) red panda
	(368, 0.00063%) hello panda
OpenImages clf (machine generated)
	(789, 0.00048%) panda
	(408, 0.00025%) red panda
RefCOCO descriptions: 22 (0.0154701%)
	Example 1: panda
	Example 2: left panda
	Example 3: center panda
	Example 4: center panda
	Example 5: right panda
OpenImages descriptions: 104 (0.0204949%)
	Example 1: In this image there is a red panda. Around the panda there are plants.
	Example 2: In the image there is a panda laying on wooden surface, beside the panda there are leaves 
	Example 3: In the image there is a panda laying on a grass and behind the panda there are few wooden 
	Example 4: In the image we can see there is a panda sitting on the ground and the panda is holding pl
	Example 5: It is a black and white image,

In [13]:
summary_report(['vacuum cleaner'])

COCO
LVIS
Objects365
ImageNet boxes
OpenImages boxes
OpenImages points
	(424, 0.00077%) vacuum cleaner
OpenImages clf (human verified)
	(1331, 0.00226%) vacuum cleaner
OpenImages clf (machine generated)
	(145, 0.00009%) vacuum cleaner
RefCOCO descriptions: 0 (0%)
OpenImages descriptions: 21 (0.00413839%)
	Example 1: In this image we can see a vacuum cleaner.
	Example 2: In this picture we can see a vacuum cleaner, bottle, brush and a long handle brush.
	Example 3: In this image I can see depiction picture where I can see a vacuum cleaner and a dog. I ca
	Example 4: In this picture I can see a human cleaning with the help of vacuum cleaner and I can see f
	Example 5: this picture shows a woman standing and holding a vacuum cleaner and cleaning the house an
Visual Genome descriptions: 63 (0.00068396%)
	Example 1: vacuum cleaner
	Example 2: A vacuum cleaner
	Example 3: A vacuum cleaner
	Example 4: Upright vacuum cleaner
	Example 5: junked vacuum cleaner
LAION 400M descriptions: 0.0124947%

In [14]:
summary_report(['shovel'])

COCO
LVIS
	(110, 0.00866%) shovel
Objects365
	(8786, 0.03458%) shovel
ImageNet boxes
	(523, 0.03528%) shovel
OpenImages boxes
OpenImages points
	(1408, 0.00255%) shovel
OpenImages clf (human verified)
	(1395, 0.00237%) shovel
	(696, 0.00118%) snow shovel
OpenImages clf (machine generated)
	(124, 0.00008%) shovel
RefCOCO descriptions: 0 (0%)
OpenImages descriptions: 41 (0.00807971%)
	Example 1: In this image we can see a shovel, rocks, plants, and grass.
	Example 2: In this picture there are plants, a shovel. At the top it is blurred.
	Example 3: In this image we can see a sand, cardboard, a net and a shovel.
	Example 4: This looks like a collage picture. I think this is the stone. This looks like a digging sh
	Example 5: In this image we can see a painting with crab, shovel and bucket with white background.
Visual Genome descriptions: 277 (0.00300725%)
	Example 1: shovel
	Example 2: shovel
	Example 3: shovel
	Example 4: shovel
	Example 5: shovel
LAION 400M descriptions: 0.00659526%)
	E

In [15]:
summary_report(['baobab'])

COCO
LVIS
Objects365
ImageNet boxes
OpenImages boxes
OpenImages points
OpenImages clf (human verified)
OpenImages clf (machine generated)
RefCOCO descriptions: 0 (0%)
OpenImages descriptions: 0 (0%)
Visual Genome descriptions: 2 (2.1713e-05%)
	Example 1: baobab trees
	Example 2: baobab trees on plain
LAION 400M descriptions: 0.00204121%)
	Example 1: The Baobab - Baobab Beach Resort & Spa
	Example 2: Orchestra Baobab: A Night at Club Baobab
	Example 3: superfruit baobab
	Example 4: stock photo of baobab  - Baobab trees in Madagascar - JPG
	Example 5: Ominous Baobab Tree
LAION COCO descriptions: 3.86282e-05%)
	Example 1: A poster for the book buried beneath the baobab tree.
	Example 2: grasslands common plants many types of grasses unique baobab trees
